# Enunciado


Este trabalho usa SageMath nas suas implementações


1. Pretende-se construir em torno de uma cifra assimétrica um conjunto de técnicas criptográficas destinadas a fins distintos. Apesar de todas as alíneas do problema poderem ser  respondidas com a maioria das cifras assimétricas clássicas ou pós-quânticas, neste problema vamos exemplificar o processo com uma técnica simples da família Diffie-Hellman nomeadamente a cifra assimétrica ElGamal com parâmetros de segurança $\,\lambda\,$.
    1. Implemente um esquema  PKE $\,\mathsf{ElGamal}(\lambda)\,$ (ver Capítulo 4) num subgrupo de ordem prima $\,q\,$,  com $\,|q|\geq \lambda\,$, do grupo multiplicativo $\,\mathbb{F}^\ast_p\,$ com $\,p\,$ um primo que verifica $\,|p| \geq \lambda\times|\lambda|$ . Identifique o gerador de chaves e os algoritmos de cifra de decifra neste esquema. Identifique o núcleo deterministico do algoritmo de cifra.
    2. Supondo que a cifra que implementou é IND-CPA segura (de novo Capítulo 4), usando a transformação de Fujisaki-Okamoto implemente um PKE que seja IND-CCA seguro.
    3. A partir de (b) construa um esquema de KEM que seja IND-CCA seguro.
    4. A partir de (b) construa uma implementação de um protocolo autenticado de "Oblivious Transfer" $\,\kappa$-out-of-$n\,$.

# **Exercício 1.a.**

EL GAMAL
Qualquer PKE é determinado por três algoritmos: geração de chaves, cifra e decifra:

 ----------<br><br>$\text{GenKeys}(\lambda)$                                 …       $\lambda\;$ é o parâmetro de segurança<br><br><br>    - gerar aleatoriamente um primo $\,q \approx 2^\lambda$                                      <br>    - gerar um primo $p$  tal que  $\,\mathbb{F}_p^\ast\,$ tem um sub-grupo de ordem $\,q\,$ ; calcular um gerador $g$ desse sub-grupo<br>    - gerar aleatoriamente  $\,0 <s < q\,$ ,  a chave privada<br>    - calcular e  revelar  a chave pública   $\,\mathsf{pk} \equiv \langle p,q, g,g^s\rangle$<br>----------<br><br>$\text{Enc}(\mathsf{pk},m)$                                   …   a mensagem $m$ é um elemento de $\mathbb{F}_p^\ast$ <br><br><br>    - obter elementos públicos  $\,p,q,g,g^s \,\gets\,\mathsf{pk}$<br>    - gerar aleatoriamente  $\,0 <\omega < q$ <br>    - calcular  $\,\gamma \gets g^\omega\;$ e $\,\kappa \gets (g^s)^\omega\,$.<br>    - construir  o criptograma $\,\mathbf{c}\gets \langle\,\gamma\,,\, m\times\kappa\,\rangle\,$<br>----------<br><br>Note-se que se verifica $\,\kappa = \gamma^s\,$.<br> 
----------<br><br>$\text{Dec}(\mathsf{sk},\mathbf{c})$  …  $\mathsf{sk} = s$ é a chave privada<br><br>
- obter a chave privada $s$<br>    
- obter o criptograma $\mathbf{c} = \langle \gamma, \delta \rangle$<br>    
- calcular $\kappa \gets \gamma^s \mod p$<br>    
- calcular $\kappa^{-1} \mod p$<br>    
- recuperar a mensagem original: $m \gets \delta \times \kappa^{-1} \mod p$<br>    

In [1]:
print_tentativas_genKeys = False

In [2]:
from sage.all import *
lambda_security = 128  # Define um tamanho de bits para q

Para encontrar um gerador do grupo multiplicativo temos que encontrar um número cuja ordem seja igual à do grupo, i.e.:
- Dada a ordem do grupo multiplicativo $F_p^* = \phi(p)$ o gerador g tem de ter ordem $\phi(p)$, ou seja, $g^{\phi(n)}=1 \ mod \ p$

In [3]:
def find_generator(p):
    """Encontra um gerador do grupo multiplicativo F_p^*."""
    if not is_prime(p):
        raise Exception("O p de input não é primo")

    phi_p = p - 1  # Para p primo, phi(p) = p - 1

    fatoracao = factor(phi_p)
    fatores_primos = list(set([q for q, e in fatoracao]))
    
    #print(f"Fatoração de phi(p): {fatoracao}")
    #print(f"Fatores primos de phi(p): {fatores_primos}")

    # Itera sobre possíveis geradores
    for g in range(2, p):
        if gcd(g, p) != 1:
            continue  # Ignora números não coprimos com p

        is_gerador = True
        for q in fatores_primos:
            if pow(g, phi_p // q, p) == 1:
                is_gerador = False
                break

        if is_gerador:
            return g 

    return None  

In [4]:
def gen_keys(lambda_security):
    print("----------------------------------Geração de chaves:--------------------------------------")
    # Gerar aleatoriamente q com bit_length maior que lambda
    q = random_prime(2^129 - 1, False, 2^128)
    print("Parâmetro q gerado:",q)
    
    #Gera-se sucessivamente inteiros pi = q*2^i+1 até que pi seja um primo suficientemente grande, ou seja |p| > 1024
    i = 1
    p_i = q * (2^i) + 1 
    tamanho_desejado = lambda_security * lambda_security.nbits()
    
    while True:
        # Caso o p_i não convergir tenta um novo q
        if p_i.nbits() > 2500:
            q = random_prime(2^129 - 1, False, 2^128)
            i = 1
            p_i = q * (2^i) + 1
            print("p não convergiu, novo valor de q: ",q)
            
        if is_prime(p_i) and p_i.nbits() >= tamanho_desejado:
            break  
    
        i += 1
        p_i = q * (2^i) + 1
        if print_tentativas_genKeys:
            print(f"Tentativa {i}: p_i = {p_i} (Tamanho: {p_i.nbits()} bits)")

    p = p_i
    print("Parâmetro p gerado:",p)
    
    # Fp é um grupo multiplicativo se para todo o x pertencente a Fp gcd(x,p) = 1 (trivial já que p é primo)
    # Criar o corpo finito F_p
    F_p = GF(p)
    # O grupo multiplicativo F_p^* é o conjunto de elementos não nulos de F_p
    F_p_star = F_p.unit_group()
        
    # Agora obtemos um gerador do subgrupo de ordem q
    g = find_generator(p)
    print("gerador (g):",g)
    
    # Gerar aleatoriamente a chave privada 0<s<q
    s = randint(1, q-1)
    print("chave privada (s):", s)
    print("---------------------------------------------------------------------------------")
    return (p, q, g, pow(g, s, p)), s

## Escolha de p:
A ordem desse grupo é $\,n = p - 1\,$ e para que o DLP seja  complexo não basta apenas que $\,p\,$ seja grande: é também necessário que o maior factor primo de $\,(p-1)\,$ seja também grande. 
Para garantir estas condições o primo $\,p\,$  é gerado de uma determinada forma:

1. Gera-se um primo $\,q\,$ grande: com mais de $\lambda$ bits de tamanho; este vai ser o maior factor de $\,(p-1)\,$
2. Gera-se sucessivamente inteiros  $\,p_i\;=\;q\,2^i + 1\,$ até que $\,p_i\,$ seja  um primo suficientemente grande .

## Como é que $F_p^*$ tem um subgrupo de ordem q? (Teorema de Lagrange)
O Teorema de Lagrange diz que, se um grupo G tem ordem finita e H é um subgrupo de G, então |H| é um divisor de |G|.

Visto que $F_p^*$ tem ordem $p-1$ e $p-1$ divide $2q$ (e consequentemente $q$) então podemos afirmar que $F_p^*$ tem um subgrupo de ordem q. 

------------------

In [5]:
def enc(pk, m):
    p, q, g, g_s = pk
    omega = randint(1, q-1)  # Escolhe um valor aleatório ω entre 0 e q
    gamma = pow(g, omega, p)  # γ = g^ω mod p
    kappa = pow(g_s, omega, p)  # κ = (g^s)^ω mod p
    c = (gamma, (m * kappa) % p) 
    return c

------------------

In [6]:
def dec(sk, pk, c):
    p, _, _, _ = pk
    gamma, delta = c
    kappa = pow(gamma, sk, p)  # κ = γ^s mod p
    kappa_inv = inverse_mod(Integer(kappa), Integer(p))  # Calcula o inverso de κ mod p
    m = (delta * kappa_inv) % p  # Recupera a mensagem original
    return m

------------------

In [7]:
# Exemplo
pk, sk = gen_keys(lambda_security)  # Geração de chaves
m = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
c = enc(pk, m)  # Cifra a mensagem
m_dec = dec(sk, pk, c)  # Decifra a mensagem

# Exibir os resultados
print(f"Mensagem original: {m}")
print(f"Criptograma: {c}")
print(f"Mensagem decifrada: {m_dec}")
print(f"Decifração bem-sucedida? {m == m_dec}")
print("---------------------------------------------------------------------------------")

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 487699698876737818552487516308232197879
Parâmetro p gerado: 1506218231047799679526744863109809970689509330025722735562989793769908233228433625215483501146216311197508334689366568906915756775533521505313352015249195953302294888544550636810076827766387213187486366751090244808198263395299431296727952282133407831525274973535471383940865746674369375861170481461652444521938571933320006321838212011172365089260051393150977
gerador (g): 3
chave privada (s): 352014016105149690688414936672900546621
---------------------------------------------------------------------------------
Mensagem original: 9932427937585554595655955559378485232797726504405600074678422624288571502784179366694889588272588459272946929591977903419159642944298895271470183786461689655996711421044824772529335169561777376398392997461690707476183329872993776434680182850108075042933815872071353587361231106288288423615352452

--------

# **Exercício 1.b.**

Supondo que a cifra que implementou é IND-CPA segura (de novo Capítulo 4), usando a transformação de Fujisaki-Okamoto implemente um PKE que seja IND-CCA seguro.

### **Transformar um  PKE-IND-CPA em um PKE-IND-CCA**

A transformação FO original constrói, a partir de $\,(E_p,D_s)\,$,  um novo esquema de cifra assimétrica $\,(E'_p,D'_s)\,$ , usando um  “hash” pseudo-aleatório $\,h\,$ de tamanho $\,\lambda\,$ e um “hash” pseudo-aleatório $\,g\,$ de tamanho $\,|x|\,$.

O algoritmo de cifra parametrizado pelos dois “hashs”  $\,h,g\,$    é 

  $$E'_{p}(x)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$$

O algoritmo $\,D'_{s}\,$ rejeita o criptograma se detecta algum sinal de fraude. 


$$D'_{s}(y,c)\;\equiv\;\vartheta\,r \gets D_s(c)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\mathsf{if}\;\;c\neq f_p(r,r')\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$

### Definições de variáveis

In [8]:
print("----------------------------------Definição de veriáveis----------------------------------")
lambda_bits = 128

pk, sk = gen_keys(lambda_bits)
x = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
length_in_bytes = (x.bit_length() + 7) // 8
x = x.to_bytes(length_in_bytes, byteorder='big')
print("length x:",len(x))
print("Mensagem de input:",x)
print("-------------------------------------------------------------------------------------------")

----------------------------------Definição de veriáveis----------------------------------
----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 678267703150883238331992576020294059763
Parâmetro p gerado: 284828789229163771187790577473635873555798367063503005037831047771526739928035601633062054324988241820076963902308682872861791497645842452824668204470992375144337691952289457013786690882949070917056684198696433598523224605390231506269050655992159110713945657007377856196348830394016751476039013789416871597674024095688258101629394343967840027070378054817390763149146847310892382957568358932691385679241501267331234017342063506721490377025814091658118206324005661550682536531153178688094008917398877429635069278177038834037808614342824711684097
gerador (g): 3
chave privada (s): 531949773555625091036396936866208983970
---------------------------------------------------------------------------------
length x: 226
Mensagem de input: b"\x0

-------

### Funções auxiliares:
- $g$, um "hash" pseudo-aleatório de tamanho $|x|$
- $h$, um "hash" pseudo-aleatório de tamanho $lambda$
- $f_p$ o núcleo determinístico da cifra ElGamal

In [9]:
import hashlib

def g(r):
    """Hash pseudoaleatório g(r) com tamanho igual ao da mensagem x"""
    g = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    g.update(r_bytes)
    final_hash = g.digest()  # Truncar para o tamanho de x
    while len(final_hash) < len(x):
        g = hashlib.sha512()
        g.update(r_bytes)
        final_hash += g.digest()
    #print("g(r) hash:", final_hash[:len(x)])
    return final_hash[:len(x)]

def h(r, y):
    """Hash pseudoaleatório h(r, y) com tamanho lambda_bits"""
    h = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    ry = bytes(a ^^ b for a, b in zip(r_bytes, y))
    h.update(ry)
    full_hash = h.digest()[:lambda_bits // 8]  # Truncar para lambda bits
    #print("h(r, y) hash:", full_hash)
    return full_hash

#Núcleo determinístico da função enc anterior
def f_p(pk, r, rlinha):
    p, q, g, g_s = pk
    gamma = pow(g, rlinha, p)  # γ = g^ω mod p
    kappa = pow(g_s, rlinha, p)  # κ = (g^s)^ω mod p
    return (gamma, (r * kappa) % p)

### Cifra IND-CPA segura 

In [10]:
print("----------------------------------CIFRA----------------------------------")
def enc_fujisaki(pk,x):
    r = ZZ.random_element(2^(lambda_bits - 1), 2^lambda_bits)
    print("r:",r)
    y = bytes(a ^^ b for a, b in zip(x, g(r)))
    rlinha = h(r,y)
    c = f_p(pk,r,int.from_bytes(rlinha,"big"))
    return (y,c)

(y,c) = enc_fujisaki(pk,x)
print("(y,c) = ",(y,c))
print("-------------------------------------------------------------------------------------------")

----------------------------------CIFRA----------------------------------
r: 238050111322519302999535069899714334886
(y,c) =  (b'u\xfa\xe4\t\xca\t\x0c\xdb\x1dv\xf8-b\x12\x87\xaec\xf5_+r\xa5I\x84n\xe3\xbb^x\x1e\x11+\x85C\xe9\xd8\x8f\x01\xed\xd8\xb0\xcc\xb03\x8aJ=S\x04\xdb\xe0%\xa9L \x07?\xa1\x19\x17\x04\xfeiF\xa8\xf8\x81\xd0\xe2\x8f\x93 \xa4wE\x9c\x06\xff[~\x1eQR-\x12\x15\x81G-;\x96>\xa0\x99@\x94\xec\xf2\xc1\x05%:\xb7\x94\x9c\xd7U\xb0\x9eb\xb6.\xff-{\x8b\xb3\x162\xc2\xf3\x00\x17\xce\xc3\xb8/f\x930\xe4Z\xe4\xd6\xafk\xae\x00\x0bz\xa2\xc3\xcd\x0en\x1c0\xc6%&\xf7\tQt\xc6Uh\xac8\x88\x84\x8b\x0ev\r\xad\xd3g\xcc\xe6\xf1\x02\xb2\x85\xac{\xe5z\x93-!\xc8\x1c\xa5\xcb8G\x0e\x13\xf8\x89\xc9\xa6\x9a\xb1*,\xe2\xd3\xd9\x1a\xd0\xba\xae\xc5K\x15\x82\xa4\x0c5\xd4\xe7\xaf\x8a\xd1\x84\x95\x08hI\\)ci\xde', (551575847270689685572702777510528378057427512037974667931419923394643068592924770292832104966147972576331698765288241217886454796583832012213595008037045775991109385430397175564588687851261271171252867814

### Decifra

In [11]:
print("----------------------------------------DECIFRA--------------------------------------------")
def dec_fujisaki(sk,pk,y,c):
    r = dec(sk,pk,c)
    print("r:",r)
    rlinha = h(r,y)
    if c != f_p(pk,r,int.from_bytes(rlinha,"big")):
        raise Exception("O criptograma não corresponde a fp(r,r'), absurdo")
    else:
        res = bytes(a ^^ b for a, b in zip(y, g(r)))
        return res

print("Mensagem decifrada: ",dec_fujisaki(sk,pk,y,c))
print("x == dec_fujisaki(sk,pk,y,c)?: ",x == dec_fujisaki(sk,pk,y,c)) 
print("-------------------------------------------------------------------------------------------")

----------------------------------------DECIFRA--------------------------------------------
r: 238050111322519302999535069899714334886
Mensagem decifrada:  b"\x02\x99\x9f\xfa\x8c\xa5MH\x1d\xae&\xb7\x1a\xc1\xbd\xe5H\x1f\xbf'\xaa8\xf0\x97\x17\x04\x96E\x07\xb43\x03\xdb0i\x808\xb7\xf8\xa2@\x0e@\xfdN\xcd\xde?\xc2\x9a!\x969\xf6\xdaeVr7\xad\x08\xb3\xf4\x16\xdf\x9b\xfa#\xa4#\xd2\xb3\xa4\xaf\x9b\x06~,a55\xbb\xb2!\xca\x888TT\xdc\xbb%\xdf3b\xbc\xb2\x81A]\x92\x8c\xa2\xeel\x15\xa5~Z\xe5UB9l\xba8#\xac\xc8\xa0\x9a\xd39t\xcf\xf5\xb26\xe4S\x9f\xa9\xa2z\xee\xf8\xae\xd8\xd5\xe0\xda\x10\xf7EE\xf6\xd0\xca\xfd\xbbN\x1a(\x93\xebN\x17\x06\x1a\xa0\xda\xf8\x8e.\xba\x1b\xc6\x1d<$\x01\xccv\x02O\x17#;R\x9e\xb1r\xe6\xc7\xa2\xebi\xb4\x1f\xb5\x14\x99\xd1\xf9\xca\xd9jN\x92J\x1a\x08d4\xbd\x98/\xc9\x8f\xe6\xd5\xd8?23\xc2\xfdr%s6\xf6\x0bK7\xad"
r: 238050111322519302999535069899714334886
x == dec_fujisaki(sk,pk,y,c)?:  True
-------------------------------------------------------------------------------------------


# **Exercício 1.c.**

Associado a um mecanismo de encapsulamento de chaves existe um algoritmo de revelação ou KRev (“_key revelation_”), que a partir do encapsulamento de uma chave, revela-a.  Naturalmente, por analogia com as cifras assimétricas, o KEM é um algoritmo  público, total e probabilístico,  enquanto que o KRev é um algoritmo privado, parcial e determinístico.

A condição de correção será
                                            $$(e,k)\,\gets\,\mathsf{KEM}\quad\;\text{sse}\;\quad \mathsf{KRev}(e)\,\simeq\, k$$

Combinando o exercício anterior com um KEM o esquema pode-se descrever pelo par de algoritmos:

$$\left\{\begin{array}{lcl}E(x) & \equiv & \vartheta\,(e,k)\gets\mathsf{KEM}\,\centerdot\,(e\,,\,k \oplus x) \\ D(e,c) & \equiv & \vartheta\,k \gets \mathsf{KRev}(e)\,\centerdot\,k\oplus c\end{array}\right.$$

Substituindo KEM pela função de cifra do exercício (b) obtemos uma segurança IND-CCA para a encapsulação da chave.
Como o KEM é um mecanismo apenas de encapsulamento da chave iremos utilizar em conjunto com um DEM (neste caso OTP) para demonstrar que resulta para cifrar uma mensagem, como é demonstrado no diagrama:

$$
\begin{array}{ccc}
\text{KEM} & \rightarrow & \text{encapsulation}  &  \rightarrow & \text{KRev} \\
\downarrow \text{key} & & & & \downarrow \text{key}\\
\oplus & \rightarrow & \text{Ciphertext} &  \rightarrow & \oplus \\
\uparrow & & & & \downarrow\\
\text{plaintext} & & & & \text{plaintext}
\end{array}
$$

----------

### Definições de variáveis

In [12]:
lambda_bits = 128

pk, sk = gen_keys(lambda_bits)
x = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
length_in_bytes = (x.bit_length() + 7) // 8
x = x.to_bytes(length_in_bytes, byteorder='big')

print("Mensagem aleatória em F_p*: ",x)

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 379967965674325107763402278673704557779
Parâmetro p gerado: 304340543531242655349448692851868462628052829515884296078656658567065342541114028136360956120201414016260770982300110613324270212888734585427747342760144055181614967508406736838278055290140797217608930311780397167465592913503293085337104704221747990978421619895835179074776312414559904935862154109630444837102332452707836006979677017315114594927616903384580103906201600121424122121127203471204653249002334734466737852217647125759372008900829807422536649219682854779225822659699222552334931546841772850199377825372549465574781672054849537
gerador (g): 3
chave privada (s): 291725272953963459071538182785024684384
---------------------------------------------------------------------------------
Mensagem aleatória em F_p*:  b'7\xb6\x128\xf9j\x06\xecb\\?\x11\xcd\xb2\xc8E\xd43E\xba5~\xb6\x80B\xbb\x05\x86N\x97L\xa8|\x8a\x1f\x89\x8a

-------

### KEM + DEM Encrypt

Construir algoritmo de cifra tal que
$ E(x) \equiv \vartheta (e,k) ← KEM⋅(e,k⊕x)  $ 
onde 
- $k$ é gerado aleatoriamente;
- $e$ é o encapsulamento dessa chave.

In [13]:
def kem_encrypt(pk, x):
    # Gerar chave simétrica aleatória k
    k = ZZ.random_element(2^(8*len(x)-1), 2^(8*len(x))).to_bytes(len(x), 'big')
    print("k: ",k)
    # Utilizar cifra do ex1.b. como KEM para encapsular k
    e = enc_fujisaki(pk,k)
    
    # Aplicar DEM (neste caso OTP) da chave gerada k com a mensagem x
    ciphertext = dem(x,k)

    return e, ciphertext

def dem(x,k):
    ciphertext = bytes(a ^^ b for a, b in zip(x, k))
    while len(ciphertext) < len(x):
        ciphertext += bytes(a ^^ b for a, b in zip(x[len(ciphertext):], k))
        ciphertext[:len(x)]
    return ciphertext

-------

### KEM + DEM Decrypt

A decifração é dada por:
$ D(e,c) ≡ ϑk←KRev(e)⋅k⊕c $
onde 
- $KRev$ no nosso caso é a técnica de decifração utilizada no exercício 1.b.;
- $e$ é o encpasulamento da chave gerada;
- $c$ o criptograma recebido de KEM.

In [14]:
def kem_dem_decrypt(sk, pk, e, ciphertext):
    (y, c) = e
    k = dec_fujisaki(sk, pk, y, c)
    print("k: ",k)
    x = dem(ciphertext,k)
    return x

---------

### Teste

In [15]:
print("-------------------------------------------------------------------------------------------------------------------")
print("mensagem (x): ",x)
e, ciphertext = kem_encrypt(pk,x)
print("-------------------------------------------------------------------------------------------------------------------")
print("e: ",e)
print("-------------------------------------------------------------------------------------------------------------------")
print("k (+) x: ",ciphertext)
print("-------------------------------------------------------------------------------------------------------------------")

mensagem = kem_dem_decrypt(sk, pk, e, ciphertext)

print("mensagem (final): ",mensagem)
print("-------------------------------------------------------------------------------------------------------------------")
print("mensagem inicial == mensagem final ?: ",x == mensagem)
print("-------------------------------------------------------------------------------------------------------------------")

-------------------------------------------------------------------------------------------------------------------
mensagem (x):  b'7\xb6\x128\xf9j\x06\xecb\\?\x11\xcd\xb2\xc8E\xd43E\xba5~\xb6\x80B\xbb\x05\x86N\x97L\xa8|\x8a\x1f\x89\x8av]!\xb7\x93\xea\xb8\xcbk\xad@Y\xe5\xee\xc4\x0b\xaa\xc3I\xde\x02\xa8\xd5\xf1\x8fC>)\xe2\xc9\xcf\xfe\xda\x0b\x11]zO\xe6\xc2<\xe5.\x90\x05\xea\xcdt2\x18ga:\xd4A\xe1\x05\xe2^><\xe7\xc4\xeeQJ\x90=\xe93E\x8f0\x8e\x1cDV\xe5\x06D\xe8\x1eb-g\x18vUpDjp\xdd\xe7R\xcc\xd23<\x16\x9c\x8cy\x80L\xaaN\xc4\xd6\xcb\x82)\xfa\x848c)Fun\xcf\xb2\x87I\x03\xda\x17\x12\xe5\x7f\xfc\xa6u\xbcn\xd5\xa8\t\xad\x1d\x16\xf2\x80\x82\x13\x81\x947\xe7\x1c\xdb\x0bE\xe2~\x8dw,\x89\xa7S_%#\xbd\x81Z\x9c\x02\x80\x98\xb8i\xfa\x8b\x19\xdah\xea\x01cNE\x9a\x93['
k:  b'\x8bC\xa4\xe4\xe7R\xc3\xa8~\xa2\xb2%\x9a\x1aP\x88j+4,1r\x8b5\xd9\t`\x9fqh\xad\x86\xb63\x96\x0e\xf8\xeaA\xb9\xb6\x14UI\xd9\xcc\xe2 \xd3\x8au?j-6>s\x12H8\xf6tP\x0bdwT\x18\xdaR\x86\x9f\x10\x1a\xee\t\xd4/J/\x13c\x89f\xef(\xc0\x85$/\x95i\x8

----------------

# Exercício 1.d.


O protocolo de “oblivious transfer”  implementa um mecanismo de transferência de informação entre dois agentes: o **Provider** (também designado por Sender) e o **Receiver** (também designado por Adversário) . Em linhas gerais, o protocolo caracteriza-se da forma seguinte:


1. O **Provider** põe à disposição para comunicação futura $\,n\,$ items de informação (ou mensagens) que ele enumera como $\,m_1, m_2, \cdots,m_{n}\;$ e que armazena de forma privada. 
    Nesta fase a única informação tornada pública é o número  de mensagens  $\,n$ .
2. O Receiver informa o **Provider** que pretende receber $\,\kappa\,$ das $\,n\,$ mensagens
3. Caso o **Provider** aceite o par $(n,\kappa)\;$ os dois agentes, a começar pelo **Provider**, trocam uma sequência de mensagens e, no final,
    1. O **Receiver** passa a conhecer exatamente  $\,\kappa\,$ mensagens mas continua a ignorar o conteúdo de todas as restantes $\,n-\kappa\,$ mensagens.
    2. O **Provider** ignora a identificação (“is oblivious of”)  das $\,\kappa\,$ mensagens que o Receiver passou a conhecer.

O protocolo usa um esquema PKE   $\,\{(E_p,D_s)\}_{(s,p)\in\mathcal{G}}$ que neste caso irão ser a cifra e decifra do exerício 1.b.. 

----

### Criterion

O critério $C{κ,n}$ define quais vetores de chaves públicas **p** são considerados válidos. O **Receiver** deve criar um vetor **p** onde:

- Algumas entradas são chaves públicas "boas" (as que correspondem às mensagens que quer receber).
- O resto das entradas são chaves públicas "más" (geradas para manter a segurança do protocolo).

O critério $\,\mathcal{C}_\kappa\,$ depende da estrutura algébrica usada pelo mecanismo de geração de pares de chaves. Vamos considerar:

- Cada chave pública, válida ou não, é codificada por um inteiro em $\,\mathbb{F}_p^*\,$.
- Uma matriz de “rank” completo  $\,\mathsf{A} \in \mathbb{F}_p^{* n\times(n-\kappa)}\,$ e um vector $\,\mathsf{u}\neq 0\in \mathbb{Z}_q^{n-\kappa}\,$ ; estes elementos são gerados por um XOF a partir de uma “seed”   $\,\rho\,$ . A  “seed” é aleatoriamente gerada e os restantes elementos são construídos com o XOF, por tentativas, até se verificarem as condições exigidas.
- O critério tem a forma de um sistema de equações lineares  $\;\mathsf{p} \times \mathsf{A}\,=\,\mathsf{u}$ .

In [16]:
import numpy as np

class CknCriterion:
    def __init__(self, kappa, n, q):
        self.kappa = kappa
        self.n = n
        self.q = int(q) 
        self.seed = np.random.randint(0, 2**32)
        self.A = self.generate_A()
        self.u = self.generate_u()
        self.Fp = GF(q).unit_group()
    
    def generate_A(self):
        """Gera a matriz A usando XOF a partir da seed"""
        np.random.seed(self.seed)
        A = random_matrix(GF(self.q), self.n, self.n - self.kappa)
        return A
    
    def generate_u(self):
        """Gera o vetor u, que deve ser não nulo"""
        np.random.seed(self.seed + 1) 
        u = vector(GF(self.q), [randint(1, self.q - 1) for _ in range(self.n - self.kappa)])
        return u
    
    def verify(self, p):
        """Verifica se p satisfaz o critério Ckn, ou seja, se p * A = u"""
        if len(p) != self.n:
            raise ValueError(f"p deve ter {self.n} elementos")
        
        p_values = [x[0] if isinstance(x, tuple) else x for x in p]
        print("p_values:",p_values)
        
        # Converter p_values para um vetor no corpo finito Z_q
        Zq = GF(self.q)
        p_vector = vector(Zq, p_values)
    
        # Calcular p * A no corpo finito Z_q
        A_matrix = matrix(Zq, self.A) 
        pA = p_vector * A_matrix 
    
        # Verificar se pA é igual a u
        u_vector = vector(Zq, self.u)  # Converter u para um vetor no corpo finito Z_q
        return pA == u_vector
    
    def print_criterion(self):
        print(f"Matriz A:\n{self.A}")
        print(f"Vetor u:\n{self.u}")

## Provider

In [17]:
class Provider:
    def __init__(self, pk, sk, n_mensagens):
        self.pk,self.sk = gen_keys(lambda_bits) #(p, q, g, pow(g, s, p)) , s
        self.numero_de_mensagens = n_mensagens
        # Informação privada das mensagens:
        self.messages = [f"mensagem{i}" for i in range(n_mensagens)]
        self.criterion = None
    def define_criterion(self,kappa):
        n = self.numero_de_mensagens
        q = pk[1]
        self.criterion = CknCriterion(kappa, n, q)

Inicialmente:
- Gera chaves
- Gera mensagens
- Expõe apenas o número de mensagens, matriz $A$ e vetor $u$

In [18]:
n_mensagens = 100
provider = Provider(None,None,n_mensagens)

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 473425505304861110036818096476948165659
p não convergiu, novo valor de q:  651747836086344281351954319461271614363
p não convergiu, novo valor de q:  601471082612323349939029982096492979499
Parâmetro p gerado: 215094375779393975774016574623634751823852143403715845903541108840829929454124348407857493305682026124217704269568959894133327956157057489001443360933372108916758307715654357662199315540180057042903829181821796856270220038252764543831080057210955536155331489709707943328114310722305101161382017909336692477010926194519409200945692023354820065462171674296319593126392304191602082328911110602149965552157982964433975343717031893968879617
gerador (g): 3
chave privada (s): 546348918973747653366047085971850553898
---------------------------------------------------------------------------------


## Receiver

In [19]:
import hashlib
from random import sample

class Receiver:
    def __init__(self, k, n_mensagens, q):
        self.k = k  # Número de chaves privadas geradas
        self.n_mensagens = n_mensagens
        self.q = q 
        self.I = sample(range(n_mensagens), k)  # Seleção aleatória de k índices
        self.e = self.enumeration(self.I)  
        self.p,self.s_values = self.generate_keys()  
        self.s = self.generate_secret()
        self.tau = self.generate_authentication_tag()

    def enumeration(self, I):
        """Cria a função de enumeração que mapeia {1, 2, ..., κ} para os elementos de I ordenados"""
        I_sorted = sorted(I)
        return {i + 1: I_sorted[i] for i in range(len(I_sorted))}

    def generate_secret(self):
        """Gera um segredo aleatório (simulado como um número grande)"""
        return ZZ.random_element(2**32)

    def generate_keys(self):
        """Gera κ chaves privadas e publicas usando o genkeys de ElGamal"""
        vetor_pk = [0] * self.n_mensagens 
        vetor_sk = []
        for i in range(1,self.k+1):
            pk,sk = gen_keys(lambda_bits) # (p, q, g, pow(g, s, p)), s
            vetor_sk.append(sk)
            vetor_pk[self.e[i]] = pk
        return vetor_pk,vetor_sk

    def generate_authentication_tag(self):
        """Gera a tag de autenticação hash(I, s)"""
        data = str(self.I) + str(self.s)
        return hashlib.sha256(data.encode()).digest()

    def complete_p_vector(self, A, u):
        """Completa o vetor p para satisfazer p * A = u no corpo finito Z_q"""
        Zq = GF(self.q)  # Define o corpo finito Z_q
    
        # Identificar os índices já preenchidos (valores diferentes de 0)
        filled_indices = [i for i in range(self.n_mensagens) if self.p[i] != 0]
        filled_values = vector(Zq, [self.p[i][0] if isinstance(self.p[i], tuple) else self.p[i] for i in filled_indices])
    
        # Criar a matriz A_filled (linhas correspondentes aos índices preenchidos)
        A_filled = matrix(Zq, [A[i] for i in filled_indices])
    
        # Criar a matriz A_empty (linhas correspondentes aos índices vazios, ou seja, onde p[i] == 0)
        A_empty = matrix(Zq, [A[i] for i in range(A.nrows()) if i not in filled_indices])
    
        # Calcular u' = u - (filled_values * A_filled)
        u_prime = vector(Zq, u) - filled_values * A_filled
    
        # Resolver o sistema linear A_empty^T * p_empty = u' no corpo finito Z_q
        try:
            A_empty_T = A_empty.transpose()
            p_empty = A_empty_T.solve_right(u_prime)
        except:
            # Se o sistema for singular, tentar solução alternativa (ex: mínimos quadrados)
            p_empty = A_empty_T.pseudoinverse() * u_prime
    
        # Preencher os elementos desconhecidos no vetor p (apenas onde p[i] == 0)
        empty_indices = [i for i in range(self.n_mensagens) if self.p[i] == 0]
        for i, idx in enumerate(empty_indices):
            # Gerar a chave "má"
            p_mau, q_mau, g_mau, gs_mau = self.gen_mau_keys() 
            self.p[idx] = (p_mau, q_mau, g_mau, gs_mau)
        return self.p
    
    def gen_mau_keys(self, max_attempts=100):
        """Gera uma chave 'má' no formato (p, q, g, g^s)"""
        attempts = 0
        while attempts < max_attempts:
            # Gerar q_mau como um número primo aleatório
            q_mau = random_prime(2^128, False, 2^127)
    
            # Tentar encontrar um k tal que p_mau = q_mau * 2^k + 1 seja primo
            for k in range(1, 10):  # Limitar k a um intervalo razoável
                p_mau = q_mau * (2^k) + 1
                if is_prime(p_mau):
                    # Criar o corpo finito F_p_mau
                    F_p_mau = GF(p_mau)
    
                    # Encontrar um gerador g_mau do grupo multiplicativo F_p_mau^*
                    g_mau = F_p_mau.multiplicative_generator()
    
                    # Gerar a chave privada s_mau aleatoriamente
                    s_mau = randint(1, q_mau - 1)
    
                    # Calcular g^s mod p
                    gs_mau = pow(g_mau, s_mau, p_mau)
    
                    return p_mau, q_mau, g_mau, gs_mau
    
            attempts += 1
    
        raise ValueError(f"Não foi possível gerar p_mau após {max_attempts} tentativas.")
    
    def print_info(self):
        print("----------------------------------------------------------------------")
        print(f"Seleção I: {self.I}")
        print("----------------------------------------------------------------------")
        print(f"Função de enumeração e: {self.e}")
        print("----------------------------------------------------------------------")
        print(f"Segredo s: {self.s}")
        print("----------------------------------------------------------------------")
        print(f"Chaves privadas s_i: {self.s_values}")
        print("----------------------------------------------------------------------")
        print(f"Vetor p (com chaves públicas mapeadas): {self.p}")
        print("----------------------------------------------------------------------")
        print(f"Tag de autenticação τ: {self.tau}")
        print("----------------------------------------------------------------------")

Escolhe o número de mensagens que pretende receber do Provider tendo acesso ao número de mensagens disponíveis

In [20]:
k = 20
receiver = Receiver(k,n_mensagens,provider.pk[1]) # Receiver escolhe o conjunto I já na sua construção

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 468601276260083369395618784167366617347
p não convergiu, novo valor de q:  456386142030968830373266851338520761167
Parâmetro p gerado: 265099183591794405915695160300324671560308897386636795244324099854230498456089622562903658082260194971043213337871005267171027971289906551786948653939748720067868608004799221595523428981705733956754552359734779162510407006530283923277871255890086278532891948604218926325204524727221793025961861544205967644736837136678913
gerador (g): 3
chave privada (s): 39822229966415039273058785286330073985
---------------------------------------------------------------------------------
----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 658316136949427614649801826651412869359
Parâmetro p gerado: 4638068330695857455560253762025612876415815945860993898444412589750059695628839660806678023230566628838380

### 1. O **Provider** gera o critério C_{k,n} e envia-o ao **Receiver**

In [21]:
provider.define_criterion(k)

### 2. O Receiver  escolhe um conjunto $\,I \subset \{1,n\}\,$ , de tamanho $\,\#I = \kappa\,$ , que identifica os índices das mensagens que pretende recolher.  
Seja $\;e\;$ a enumeração de $\,I\,$:  a função crescente $\,e\colon\{1,\kappa\}\to \{1,n\}\;$  cuja imagem é $\,I\,$.

O Receiver compromete-se com a escolha de mensagens da seguinte forma (dado o conjunto $I$ e a função crescente $e$):

1. Gera aleatoriamente um segredo $\,\mathbf{s}\,$ e , usando um XOF com$\,\mathbf{s}\,$ como “seed”, constrói$\,\kappa\,$  chaves privadas$\,s_1,\cdots,s_\kappa\,$ .
2. Para cada$\,i\in \{1,\kappa\}\,$ gera chaves públicas $\;\upsilon_i \gets \mathsf{pk}(s_i)\,$ e atribui o valor$\,\upsilon_i\,$ à componente de ordem$\,e(i)\,$ do vector $\,\mathsf{p}\;$; ou seja , executa$\,$ $\,\mathsf{p}_{e(i)} \gets \upsilon_i\,$
3. Gera uma “tag” de autenticação$\,$para a seleção$\,I\,$ e o segredo$\,\mathbf{s}\,$

$$\,\tau \gets \mathsf{hash}(I\,,\,\mathbf{s})\,$$

(Definido na classe:)

In [22]:
receiver.print_info()

----------------------------------------------------------------------
Seleção I: [7, 74, 80, 57, 2, 29, 3, 20, 4, 67, 15, 28, 10, 98, 40, 35, 61, 8, 55, 64]
----------------------------------------------------------------------
Função de enumeração e: {1: 2, 2: 3, 3: 4, 4: 7, 5: 8, 6: 10, 7: 15, 8: 20, 9: 28, 10: 29, 11: 35, 12: 40, 13: 55, 14: 57, 15: 61, 16: 64, 17: 67, 18: 74, 19: 80, 20: 98}
----------------------------------------------------------------------
Segredo s: 1626447042
----------------------------------------------------------------------
Chaves privadas s_i: [39822229966415039273058785286330073985, 343347144649756448864458167347865142323, 353368548385613739706645266031680521474, 255557811927045804038532781631186778513, 198341130538577325603860882039892100043, 64469076890246090008274633860416453612, 70317692219282548406715057325397986923, 493483774982927187919444285799807421619, 74395228704494580996344400193131340377, 57531422204828047005062251041721297926, 313840771

Em seguida completa a definição de $\,\mathsf{p}\,$ atribuindo às compompentes $\,\{\mathsf{p}_j\}_{j\in\!\!\!/I}$valores tais que o vetor de chaves públicas  $\,\mathsf{p}\,$ seja aceite pelo critério $\,\mathcal{C}_{\kappa,n}\,$.

Para que o vetor **p** satisfaça a equação $p×A=u$, onde **A** é a matriz gerada pelo critério $C_{k,n}$ e **u** é o vetor correspondente, o Receiver precisa preencher os espaços em **p** que não foram preenchidos pelas chaves públicas de forma que a equação seja satisfeita.

Mais concretamente essa forma de completar o vetor é descrita na classe Receiver:

O código realiza a seguinte sequência de operações para completar o vetor p de forma que satisfaça a equação p×A=up×A=u no corpo finito $Z_q$:

- Definição do corpo finito $Z_q$ usando ```GF(self.q)```.

- Identificação dos índices preenchidos no vetor $p$ usando ```filled_indices```, que são obtidos a partir dos valores em ```self.e```. Os valores correspondentes a esses índices são armazenados em ```filled_values```.

- Criação das matriz ```A_filled```  partir das linhas da matriz $A$ correspondentes aos índices preenchidos.

- Criação das matriz ```A_empty``` a partir das linhas da matriz $A$ correspondentes aos índices vazios.

- Cálculo do vetor $u′$ subtraindo a contribuição dos elementos preenchidos de $u$, ou seja, ```u′ = u − (filled_values × A_filled)```.

- Resolução do sistema linear $p_{empty} = A_{empty}^T . u′$ para preencher os valores desconhecidos de $p$ (chaves "más").

- Preenchimento de p, colocando os valores de $p_{empty}$ índices correspondentes no vetor p.

Finalmente o Receiver envia ao Provider a “tag” $\,\tau\,$ e o vetor $\,\mathsf{p}\,$

In [23]:
completed_p = receiver.complete_p_vector(provider.criterion.A, provider.criterion.u)
tau = receiver.tau

### 3. O Provider  determina  $\,\mathcal{C}_{\kappa,n}(\mathsf{p})\,$; se $\,\mathsf{p}\,$ não for aceite pelo critério então aborta o protocolo.

Se $\,\mathsf{p}\,$  for aceite,  então usa  a variante IND-CCA  da cifra IND-CPA com a “tag”  $\tau$

$$E'_{p}(x,\tau)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y,\tau)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$$

cuja característica específica é o facto de se incluir o “tag” $\,\tau\,$ no “hash”  $\;h(r,y,\tau)\;$usado para construir a pseudo-aleatoriedade $\,r'\,$.

Usando esta cifra o Provider  constrói $\,n\,$ criptogramas 
$$\;(y_i,c_i) \gets E'_{p_i}(m_i)\quad$$  com $\,i\in\{1,n\}$
que envia para o Receiver.

Iremos definir um novo "hash" pseudo-aleatório $h$ para que consiga receber como argumento a tag $\tau$:

In [24]:
def h_OT(r,y,t):
    """Hash pseudoaleatório h(r, y, t) com tamanho lambda_bits"""
    h = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    ry = bytes(a ^^ b for a, b in zip(r_bytes, y))
    ryt = bytes(a ^^ b for a, b in zip(ry, t))
    h.update(ryt)
    full_hash = h.digest()[:lambda_bits // 8]  # Truncar para lambda bits
    #print("h(r, y) hash:", full_hash)
    return full_hash

E definir a cifra, que na realidade irá ser a do exercício 1.b. mas com um argumento extra $\tau$:

In [25]:
def f_p_OT(pk, r, rlinha):
    p, q, g, g_s = pk
    # Verificar se p é primo
    if not is_prime(p):
        raise ValueError(f"p = {p} não é um número primo. Não é possível criar o corpo finito GF(p).")


    # Calcular gamma e kappa
    gamma = pow(g, rlinha, p)  # γ = g^ω mod p
    kappa = pow(g_s, rlinha, p)  # κ = (g^s)^ω mod p

    # Multiplicar r_fp por kappa no corpo finito F_p
    result = (r * kappa) % p

    return (gamma, result)

In [26]:
def enc_fujisaki_OT(pk,x,t):
    # r gerado aleatoriamente com lambda bits
    r = ZZ.random_element(2^(lambda_bits - 1), 2^lambda_bits)

    # y = x XOR g(r)
    y = bytes(a ^^ b for a, b in zip(x, g(r)))

    # r' = h(r,y,tau)
    rlinha = h_OT(r,y,t)
    print(rlinha)

    # c gerado a partir do núcleo determinístico 
    c = f_p_OT(pk,r,int.from_bytes(rlinha,"big"))

    #Devolver criptograma
    return (y,c)

Determinamos então $C_{k,n}(p)$, se este for aceite ciframos todas as mensagens com as chaves públicas fornecidas em $p$:

In [27]:
import sys 
if not provider.criterion.verify(completed_p):
    print("------------------------------------------------------------------------")
    print("O vetor p foi aceite")
    print("p final:[ ")
    for key in completed_p:
        print(key)
    print("]")
    print("------------------------------------------------------------------------")
    ciphertext_vector = []
    i = 0
    for public_key in completed_p:
        if isinstance(public_key, tuple):
            byte_length = (sys.getsizeof(provider.messages[i]))
            ciphertext_vector.append(enc_fujisaki_OT(public_key, bytes(provider.messages[i], 'utf-8'), tau))
            i += 1
        else:
            raise ValueError(f"encontrada chave pública não primo: {public_key}")
else:
    print("O vetor p não foi aceite. Abortado")

p_values: [16346905232881361996093839879029244433729, 2466501409751646836037369762551332052297, 265099183591794405915695160300324671560308897386636795244324099854230498456089622562903658082260194971043213337871005267171027971289906551786948653939748720067868608004799221595523428981705733956754552359734779162510407006530283923277871255890086278532891948604218926325204524727221793025961861544205967644736837136678913, 4638068330695857455560253762025612876415815945860993898444412589750059695628839660806678023230566628838380282241293229419815295937619150648377261532576537590105987779439922930759574490770573644523361810138338669414830286330637329084187971314571036596761440673838572913941946539350466494638351415243720482250790115658748328147652033297624440828153875930459551238795021925078462094466962477089148686478111565147156056183324295260288394806904642041151133782312382114467596742823104019288138697279998600286242000091897045383241271532284279293356957237249, 23060699605857076360253260370

### 4. O Receiver  usa  a variante IND-CCA  da cifra IND-CPA com a “tag” de autenticação $\tau$
$$D'_{s}(y,c,\tau)\;\equiv\;\vartheta\,r \gets D_s(c)\,\centerdot\,\vartheta\,r'\gets h(r,y,\tau)\,\centerdot\,\mathsf{if}\;\;c\neq f_p(r,r')\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$
         
uma vez mais a única característica particular deste algoritmo é o uso da “tag” de autenticação $\,\tau\,$ na construção da pseudo-aleatoriedade $\;r'\gets h(r,y,\tau)\;$.


O agente Receiver 
   - conhece, porque criou,  a “tag” $\,\tau\,$que autentica o conjunto de mensagens escolhidas $\,I\,$ e  o respetivo conjunto de chaves públicas (as “boas” chaves).
    - conhece , porque gerou e armazenou num passo anterior,  as chaves privadas $\,s_i\,$ para todos $\,i\in I\,$  
    - conhece, porque recebeu do Receiver,  todos os criptogramas $\{(y_i,c_i)\}_{i\in\{1,n\}}\,$
        
Então,  para todo $\,i\in I\,$,  pode recuperar  a mensagem
                                                        $$m_i \,\gets\, D_{s_i}(y_i,c_i,\tau)$$ 


Como na cifra vamos fazer com que a decifra possa receber a tag de autenticação:

In [28]:
def dec_fujisaki_OT(sk,pk,y,c,tau):
    r = int(dec(sk,pk,c))
    rlinha = h_OT(r,y,tau)
    if c != f_p_OT(pk,r,int.from_bytes(rlinha,"big")):
        print("má decifração")
        return None
    else:
        print("boa decifração")
        res = bytes(a ^^ b for a, b in zip(y, g(r)))
        return res

In [29]:
# Decifrar e verificar as mensagens
decrypted_messages = []
for msg_number, idx in receiver.e.items(): 
    if idx < len(ciphertext_vector):  # Verifica se o índice é válido
        print(f"\nNúmero da mensagem: {msg_number}")
        print(f"Índice no vetor ciphertext_vector: {idx}")
        print(f"Chave privada (sk): {receiver.s_values[msg_number - 1]}")  # Ajuste para índice base 0
        print(f"Chave pública (pk): {receiver.p[idx]}")
        print("len(ciphertext_vector): ",len(ciphertext_vector))
        # Verifica se a mensagem foi cifrada corretamente
        if ciphertext_vector[idx] is None:
            print("idx:",idx)
            print(f"Mensagem {idx} não foi cifrada corretamente.")
            continue

        y, c = ciphertext_vector[idx]
        sk = receiver.s_values[msg_number - 1] 
        decrypted_message = dec_fujisaki_OT(sk, receiver.p[idx], y, c, tau)

        if decrypted_message is not None:
            print(f"Mensagem decifrada: {decrypted_message}")

            # Compara com a mensagem original no Provider
            if decrypted_message == provider.messages[idx].encode('utf-8'):
                print("Decifração bem-sucedida! A mensagem decifrada corresponde à original.")
                decrypted_messages.append(decrypted_message)
            else:
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                print("Erro na decifração: A mensagem decifrada NÃO corresponde à original.")
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        else:
            print("Erro na decifração intencional.")


Número da mensagem: 1
Índice no vetor ciphertext_vector: 2
Chave privada (sk): 39822229966415039273058785286330073985
Chave pública (pk): (265099183591794405915695160300324671560308897386636795244324099854230498456089622562903658082260194971043213337871005267171027971289906551786948653939748720067868608004799221595523428981705733956754552359734779162510407006530283923277871255890086278532891948604218926325204524727221793025961861544205967644736837136678913, 456386142030968830373266851338520761167, 3, 103008136187908351401720311287907291050006374079978479625359689173550807975371698533359150948566332021851875953051212709069913707057166678411145050642516825877517803696196965081311183982258480330706032856417236547902690706297352018358863361831388341499795264188683821823072220050044818253626035899343524348834340984251904)
len(ciphertext_vector):  100
boa decifração
Mensagem decifrada: b'mensagem2'
Decifração bem-sucedida! A mensagem decifrada corresponde à original.

Número da mensagem: 2
